In [ ]:
from pathlib import Path

In [ ]:
with open("input_day7.txt") as f:
    lines = [line.rstrip("\n") for line in f.readlines()]

In [ ]:
lines

In [ ]:
commands = [e for e in lines if e != "$ ls"] # get rid of ls commands - not used for anything
commands[0] = "$ cd root" # just to make it easier to look at

In [ ]:
# verify that the only things left in commands other than cd and dir are actual files
[e for e in commands if "$ cd" not in e and "dir" not in e]

In [ ]:
def get_directory_structure(commands):
    current_level = Path("")
    directory_structure = {} 
    # keys in this object will be every single directory/subdirectory on the disk, 
    # values will be lists of other directories (as strings) and files (as a dictionary - name: size)
    
    for c in commands: 
        
        # if cd, we're moving into a new directory
        if c.startswith("$ cd"):
            # going up one level
            if ".." in c:
                current_level = current_level.parent
            # going down one level
            else:
                dir_name = c.split()[2] # splitting on whitespace gets [$, cd, dir_name] so just take last element
                current_level = current_level / dir_name
                level = str(current_level.as_posix()) # make the path object into a string
                # if we haven't been here before, add the current path into the directory structure
                if level not in directory_structure.keys():
                    directory_structure[level] = [{}] # empty dictionary which will store all files in this folder
        
        # if dir, it means the current directory contains some more directories so they should be added to its list of contents 
        elif c.startswith("dir"):
            dir_name = c.split()[1] # splitting on whitespace gets [dir, dir_name] so just take last element
            level = str(current_level.as_posix())
            directory_structure[level].append(str((current_level / dir_name).as_posix()))
        
        # since we stripped out ls commands at the start, anything else must be a file
        else:
            size, filename = c.split()
            level = str(current_level.as_posix())
            directory_structure[level][0][filename] = int(size)
    
    return directory_structure

In [ ]:
get_directory_structure(commands)

In [ ]:
directory_structure = get_directory_structure(commands)

In [ ]:
def get_directory_size(directory, directory_structure):
    total = 0
    
    for e in directory_structure[directory]:
        # if the element is a dictionary, it's the files contained directly in this directory - sum their sizes
        if isinstance(e, dict):
            total += sum([v for v in e.values()])
        # if the element is a string, it's a subdirectory - sum the size of all the files in there
        elif isinstance(e, str):
            total += get_directory_size(e, directory_structure)
    
    return total

In [ ]:
def solution_p1(directory_structure, max_size=100000):
    
    total = 0
    
    for d in directory_structure:
        size = get_directory_size(d, directory_structure)
        if size <= max_size:
            total += size

    return total

In [ ]:
solution_p1(directory_structure)

The total disk space available to the filesystem is `70000000`. To run the update, you need unused space of at least `30000000`. You need to find a directory you can delete that will free up enough space to run the update.

Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update. What is the total size of that directory?

In [ ]:
available_space = 70000000 - get_directory_size('root', directory_structure)
need_to_free_up = 30000000 - available_space

In [ ]:
print(available_space)
print(need_to_free_up)

In [ ]:
directory_sizes = {}
    
for d in directory_structure:
    directory_sizes[d] = get_directory_size(d, directory_structure)

In [ ]:
directory_sizes

In [ ]:
# Part 2
min([v for v in directory_sizes.values() if v >= need_to_free_up])